# Pipl для разбиения на чанки

##   Импорт библиотек



In [ ]:
import os  # работа с файловой системой
import re  # регулярные выражения
import math  # математические функции
import requests  # HTTP-запросы
from urllib.parse import unquote  # декодирование URL
from pathlib import Path  # работа с путями
from zipfile import ZipFile  # работа с ZIP-архивами
from hashlib import md5  # генерация MD5-хешей
import pandas as pd  # работа с таблицами

##  Список ссылок

Создаем список ссылок на документы с сайта Ранха

In [ ]:

document_links = [
    "https://www.ranepa.ru/upload/doc/prikazy-ranhigs/%D0%9F%D0%BE%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D1%82%D0%B2%D0%B0%20%D0%A0%D0%A4%20%D0%BE%D1%82%2012%20%D0%BC%D0%B0%D1%8F%202012%20%D0%B3%20N%20473%20%D0%9E%D0%B1%20%D1%83%D1%82%D0%B2%D0%B5%D1%80%D0%B6%D0%B4%D0%B5%D0%BD%D0%B8%D0%B8%20%D1%83%D1%81%D1%82%D0%B0%D0%B2%D0%B0.pdf",
    "https://www.ranepa.ru/upload/doc/accred/vip-19.03.2025.zip",
    "https://www.ranepa.ru/upload/doc/accred/vip-accred-20.05.2025.zip",
    "https://www.ranepa.ru/upload/doc/pk/2025/Pravila_priema_bac_spec_2025.pdf",
    "https://www.ranepa.ru/upload/doc/pk/2025/PP-2025-pril-1.pdf",
    "https://www.ranepa.ru/upload/doc/pk/2025/min_ball_2025.pdf",
    "https://www.ranepa.ru/upload/doc/pk/2025/min_ball_2025_fill.pdf",
    "https://www.ranepa.ru/upload/doc/pk/2025/PP-2025-pril-2_fil.pdf",
    "https://www.ranepa.ru/upload/doc/pk/2025/Prilogenie_3.pdf",
    "https://www.ranepa.ru/upload/doc/pk/2025/Prilogenie_4_Osobennosti_VI.pdf",
    "https://www.ranepa.ru/upload/doc/prikazy-ranhigs/Pravila_priema_RANHiGS_2021_reglament_inv_ED.pdf",
    "https://www.ranepa.ru/upload/doc/prikazy-ranhigs/Pologenie_Priemnoi_komisii_2024.pdf",
    "https://www.ranepa.ru/upload/doc/prikazy-ranhigs/Pologenie__exam_apell_komиссии.pdf",
    "https://www.ranepa.ru/upload/doc/prikazy-ranhigs/Pologenie_vstupitelnoe_ispитание_2023.pdf",
    "https://www.ranepa.ru/upload/doc/prikazy-ranhigs/Pravila_Appeal_2023.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/273-zакон_2020.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/FZ_Ob_obразовании_v_RF_priem.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/Federalный_zakon_19-FZ.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/152-FZ.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/Ukaz_27.04.2023_N_307.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/%D0%9F%D0%BE%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D1%82%D0%B2%D0%B0%20%D0%A0%D0%A4%20%D0%BE%D1%82%2027.04.2024%20N%20555%20%D0%B2%20%D1%80%D0%B5%D0%B4.%20%D0%BE%D1%82%2007.04.2025.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/Pravila_mat_kapital.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/Postanovlenie_89.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/Poryadok_priema.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/Prikaz_233_552_04.04.2023.pdf",
    "https://www.ranepa.ru/upload/doc/prikazy-ranhigs/Perechen_VI_27.11.2024_820.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/Prikaz_571.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/Prikaz_620.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/Prikaz_17_02_2025_N_107.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/243_02.03.2023.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/Prikaz_876.pdf",
    "https://www.ranepa.ru/upload/doc/pk/npa/2025/Prikaz_219.pdf",
]

#создаем здесь словарик, в который добавим ссылки на документы (потом используем в итоговой табице с чанками)
filename_to_url = {}
for url in document_links:
    fname = unquote(Path(url).name)
    filename_to_url[fname] = url



## Сохранение документов

 Здесь скачиваем документы по ссылкам из списка, сохраняем их в директорию SAVE_DIR внутри юпитера.

In [ ]:

SAVE_DIR = Path("ranepa_documents")
SAVE_DIR.mkdir(exist_ok=True)


def download_all():

    for url in document_links:
        fname = unquote(Path(url).name)
        path = SAVE_DIR / fname

        # Пропускаем, если скачан ранее
        if path.exists():
            print(f" Уже скачано: {fname}")
            continue

        try:
            resp = requests.get(url, timeout=60)
            resp.raise_for_status()
            path.write_bytes(resp.content)
            print(f" Скачан: {fname}")

            # Если файл — ZIP, распакуем в папку с его именем (без расширения)
            if fname.lower().endswith(".zip"):
                extract_to = SAVE_DIR / path.stem
                extract_to.mkdir(exist_ok=True)
                with ZipFile(path, "r") as zf:
                    zf.extractall(extract_to)
                print(f" Распакован: {fname} → {extract_to}")

        except Exception as e:
            print(f" Не удалось скачать {fname}: {e}")



##  Конвертация PDF / DOCX / HTML

Здесь с помощью PyPDF2 извлекаем текст со всех страниц (страницы, которые не содержат текста, заменяем на пустые строки). Также с помощью python-dox извлекаем текст из DOCX-параграфов. В зависимости от суффикса файла возвращаем чистый текст (для каждого суффикса свой метод)

In [ ]:

from PyPDF2 import PdfReader
from docx import Document as DocxDocument


def pdf_to_text(fp: Path) -> str:

    reader = PdfReader(str(fp))
    text_pages = [page.extract_text() or "" for page in reader.pages]
    return "\n".join(text_pages)


def docx_to_text(fp: Path) -> str:

    doc = DocxDocument(str(fp))
    return "\n".join(p.text for p in doc.paragraphs)


def plain_text(fp: Path) -> str:

    suffix = fp.suffix.lower()
    if suffix == ".pdf":
        return pdf_to_text(fp)
    if suffix == ".docx":
        return docx_to_text(fp)
    if suffix in {".txt", ".html", ".htm"}:
        return fp.read_text(encoding="utf-8", errors="ignore")
    return ""




## Смысловое разбиение (TF‑IDF + кластеризация)

Здесь прописываем функции, при помощи которых будем разбивать на чанки:
- Разбиваем текст на предложения (сплит при помощи точек, вопросительных/воскрицательных знаков и пробелов
- Векторизуем полученные предложения при помощи TfidfVectorizer(max_features=5000):
- Находим нулевые векторы (если в предложении есть стоп слова или токены не вошли в словарь):
    - Если все векторы оказываются нулевыми - просто возвращаем один чанк, состоящий из всего текста.
    - Иначе кластеризуем только ненулевые векторы, используя агломеративную кластеризацию. Составляем список меток: для нулевых векторов метка 1, для остальных в зависимости от кластеризации. Проходимся по предложениям и склеиваем соседние, у которых метка совпадает

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering


def split_sentences(text: str):

    parts = re.split(r"(?<=[.!?])\s+", text)
    return [p.strip() for p in parts if p.strip()]


def semantic_chunks(sentences, max_sent_per_chunk=5):

    if not sentences:
        return []

    # TF‑IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    mat = vectorizer.fit_transform(sentences)
    dense = mat.toarray()  # shape = (n_sentences, n_features)

    # Определяем, какие строки — полностью нулевые
    zero_idx = [i for i, row in enumerate(dense) if not row.any()]
    nonzero_idx = [i for i in range(len(sentences)) if i not in zero_idx]

    # Если все предложения — «нулевые» → возвращаем один чанк (join)
    if len(nonzero_idx) == 0:
        return [" ".join(sentences)]

    # Определяем число кластеров
    n_clusters = max(1, math.ceil(len(nonzero_idx) / max_sent_per_chunk))

    if n_clusters >= len(nonzero_idx):
        return [" ".join(sentences)]

    #Кластеризуем только ненулевые
    sub_mat = dense[nonzero_idx]
    model = AgglomerativeClustering(
        n_clusters=n_clusters,
        affinity="cosine",
        linkage="average"
    ).fit(sub_mat)
    sub_labels = model.labels_

    #Восстанавливаем полный список меток длины len(sentences):
    labels = [-1] * len(sentences)
    for idx, lab in zip(nonzero_idx, sub_labels):
        labels[idx] = lab

    #Склеиваем предложения с одинаковой меткой
    chunks = []
    buf = [sentences[0]]
    cur_label = labels[0]
    for sent, lab in zip(sentences[1:], labels[1:]):
        if lab == cur_label:
            buf.append(sent)
        else:
            chunks.append(" ".join(buf))
            buf = [sent]
            cur_label = lab
    if buf:
        chunks.append(" ".join(buf))

    return chunks



## Наш пайплайн

Собираем пайплайн, получаем итоговый датафрейм:
- Проходим по всем файлам в сохраненной директории.
    - Для каждого файла:
         - извлекаем текст,
         - разбиваем текст на предложения,
         - собираем чанки,
         - чистим чанки (удаляем слишком маленькие, дупликаты, убираем номера страниц и ненужные символы)
         - для каждого чанкa готовим словарь со столбцами:
           {
             "doc_id":   первые 8 символов MD5 от fp.name,
             "chunk_id": порядковый номер чанка в этом документе,
             "title":    название документа,
             "text":     текст чанка,
             "url":      ссылка на документ,
             "doc_type": "Нормативный акт",
             "length":   количество символов
           }
    - Собираем полученные словари в list `rows`.
    - Конвертируем rows в датафрейм и сохраняем в CSV

In [ ]:

def pipeline(limit=None):

    rows = []
    files = list(SAVE_DIR.rglob("*"))
    if limit:
        files = files[:limit]

    for fp in files:
        if not fp.is_file():
            continue

        text_raw = plain_text(fp)
        if not text_raw.strip():

            continue

        sentences = split_sentences(text_raw)
        if not sentences:
            continue

        # Получаем чанки
        chunks = semantic_chunks(sentences, max_sent_per_chunk=5)
        # Генерируем уникальный doc_id
        doc_id = md5(fp.name.encode()).hexdigest()[:8]
        # Читаемый заголовок
        title = fp.stem.replace("_", " ")
        # Получаем ссылку из словаря со второй ячейки
        url = filename_to_url.get(fp.name, "")
        # "Нормативный акт :D)
        doc_type = "Нормативный акт"

        for i, chunk in enumerate(chunks):
            # Чистим, чистим и еще раз чистим
            lines = chunk.splitlines()
            cleaned_lines = []
            for line in lines:
                s = line.strip()
                if re.fullmatch(r"\d+", s):
                    continue
                if re.fullmatch(r"(Стр\.?|Страница)\s*\d+", s, flags=re.IGNORECASE):
                    continue
                cleaned_lines.append(s)
            text_clean = " ".join(cleaned_lines)

            # Скипаем, если после чистки мало символов
            if len(text_clean) < 100:
                continue

            rows.append({
                "doc_id":   doc_id,
                "chunk_id": i,
                "title":    title,
                "text":     text_clean,
                "url":      url,
                "doc_type": doc_type,
                "length":   len(text_clean)
            })

        print(f"    Обработан: {fp.name} → {len(chunks)} чанков (до фильтрации длиной)")

    # Формируем DataFrame и сохраняем CSV
    df = pd.DataFrame(rows)
    OUT_DIR = Path("data/processed")
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    out_csv = OUT_DIR / "ranepa_semantic_chunks_full.csv"
    df.to_csv(out_csv, index=False, sep=";", quotechar='"', encoding="utf-8-sig")
    print(f"\n Готово: {out_csv}  ({len(df)} строк всего)")




if __name__ == "__main__":
    pipeline(limit=None)

C:\Users\gserb\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


    Обработан: 152-FZ.pdf → 547 чанков (до фильтрации длиной)


C:\Users\gserb\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


    Обработан: 273-zakon_2020.pdf → 3101 чанков (до фильтрации длиной)


C:\Users\gserb\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


...

C:\Users\gserb\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


    Обработан: Pravila_priema_RANHiGS_2021_reglament_inv_ED.pdf → 50 чанков (до фильтрации длиной)


C:\Users\gserb\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


    Обработан: Prikaz_17_02_2025_N_107.pdf → 3099 чанков (до фильтрации длиной)


C:\Users\gserb\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
